In [81]:
import re
import ast
import os
import keyword
from nltk.tokenize import RegexpTokenizer
from nltk.corpus import stopwords  
from nltk.tokenize import word_tokenize
from openpyxl import Workbook
import pandas as pd
import astunparse
import openpyxl


In [82]:
##### Class is to store the hold each and every code snippet of particular code#####
class Module:
    def __init__(self,module_code,comments):
        self.module_code= module_code
        self.source_file=None
        self.comments=comments
        self.args=None
        self.fun_call=None
        
        

In [83]:
def get_description(script):
    """ function to extract description from the docstring of the codesnippet"""
    detail = script.lower()
    detail_head_tags=re.findall("[\s\t]+[.][A-Za-z]+",detail)
    detail_text=re.split("[\s\t]+[.][A-Za-z]+\n",detail)
    detail_head_tags=[re.sub("[\n\s\t.]","",x) for x in detail_head_tags]

    #print(detail_head_tags)
    #print(detail_text)
    index=detail_head_tags.index("description")
    desc = detail_text[index+1]
    desc = ' '.join(desc.split())
    return desc

In [84]:
def check_valid_comment(pattern1,pattern2,string):
    """ to check whether the passed string can be used as a comment to describe the code snippet """
    x=re.findall(pattern1,string)
    if len(x) != 0:
        x=re.findall(pattern2,x[0])
        x=x[0]
        x=x[1:len(x)-1]
        y = x.split()
        if len(y) >= 3:         
            return x
    return False

def get_comment_line_nos(code):
    """  function to locate the comments and writelogs present in the code """
    i=1
    comment_lines = {}
    for line in code.splitlines():
        #print(line)
        temp=line
        line_no=None
        comment=None
        code_prescence="comment"
        if re.findall("#+.*",temp):
            print("over here")
            temp=re.findall("#+.*",temp)
            if len(temp)>=1:
                line_no=i
                comment = re.sub('\s*#+\s*','',temp[0])
                comment = re.sub('\s*"+\s*','',comment)
        
       
        if(line_no):
            temp=[comment,code_prescence]
            #temp[0]=remove_escape_chars(temp[0])
            comment_lines[line_no]=temp
        i=i+1
    print(comment_lines)
    return comment_lines
        
        

In [85]:
def check_module(node):
    """ check the passed node can be considered as code snippet """
    check_list=[ast.FunctionDef,ast.AsyncFunctionDef,ast.For,ast.AsyncFor,ast.While,ast.If,ast.With,ast.AsyncWith,ast.Try]
    if hasattr(node,'lineno') and hasattr(node,'end_lineno'):
        for x in check_list:
            if isinstance(node,x):
                return True
    return False

def separate_modules(code):
    """ separate the code in to different code snippets(outer blocks are only considered)"""
    data=ast.parse(code)
    parts=[]
    for node in ast.walk(data):
        if check_module(node) :
            parts.append([node.lineno,node.end_lineno])
    parts2=[i for i in parts]
    
    print(parts2)
    print(parts)
    for temp in parts2:
        for temp2 in parts:
            if temp != temp2 and temp[0] >= temp2[0] and temp[1] <= temp2[1]:
                if temp in parts:
                    print("here",temp)
                    parts.remove(temp)

    return parts
string=""" 
def func1(a,b):
    # this function just adds two numbers
    temp = a+b
    return temp

#this logic checks which number is smallest
a=10
b=20
if b<a:
    print("b is smaller")
else:
    print("a is smaller")
    
def find_A_lessthan_B(a,b):
    # This function checks if the given input a is less than b or not
    if a<b:
        return True
    else:
        return False
    
#to check the smallest of all three numbers
c=100
if find_A_lessthan_B(a,b):
    #we have to check smallest of other two numbers
    if find_A_lessthan_B(a,c):
        print("a is smaller")
    else:
        print("c is smaller")
elif find_A_lessthan_B(b<c):
    print("b is smaller")

else:
    print("c is smaller")
        
"""
get_comment_line_nos(string)

over here
over here
over here
over here
over here
{3: ['this function just adds two numbers', 'comment'], 7: ['this logic checks which number is smallest', 'comment'], 16: ['This function checks if the given input a is less than b or not', 'comment'], 22: ['to check the smallest of all three numbers', 'comment'], 25: ['we have to check smallest of other two numbers', 'comment']}


{3: ['this function just adds two numbers', 'comment'],
 7: ['this logic checks which number is smallest', 'comment'],
 16: ['This function checks if the given input a is less than b or not',
  'comment'],
 22: ['to check the smallest of all three numbers', 'comment'],
 25: ['we have to check smallest of other two numbers', 'comment']}

In [86]:
def remove_escape_chars(s):
    """ function to remove the escape characters """
    escapes = ''.join([chr(char) for char in range(1, 32)])
    translator = str.maketrans('', '', escapes)
    return s.translate(translator)

In [87]:
path=".\\Repo"

In [88]:
def count_spaces(line):
    """ counts no of spaces present in the given string """
    count=0
    for y in line:
        if(y==" "):
            count=count+1
        else:
            break
    return count
            

In [89]:
def check_module_code(code):
    """ ignore the code snippet which have more ignore_modules compared to total lines of code """
    ignore_module = ["check_mandatory_vars","args.","parser.","import","argparse.","os.path.realpath"]
    code_lines=code.split("\n")
    #print("checking here",code)
    print("code =",code_lines,"\n length=",len(code_lines))
    for x in ignore_module:
        count=code.count(x)
        if  x in code and (len(code_lines) - count >1):
            print(x,"here2")
            return False
    return True
string="""##### GETTING COMMANDLINE PARAMETERS #####

parser = argparse.ArgumentParser()
parser.add_argument('--serverName')
parser.add_argument('--username')
parser.add_argument('--password')"""
print(check_module_code(string))

def create_module(all_comments,module_lines,script):
    """ code to segregate the comments,code together and give output as one module. in the end it will be returned as list of modules  """
    i=1
    total_modules_in_script=[]
    
    script_lines=script.splitlines()
    n=len(script_lines)
    comments=[]
    code_lines=[]
    while(i<=n):
        if (i in all_comments.keys() and count_spaces(script_lines[i-1])==0) or (i==len(script_lines) and len(comments)!=0):
            if(len(code_lines)>3 and len(comments)!=0):
    
                temp2=code_lines[0] if code_lines[0]<comments[0] else comments[0]
                temp_module_code="\n".join(script_lines[temp2-1:i-1])
                if check_module_code(temp_module_code):
                    #print("\n\n$here =",temp_module_code)
                    temp3=[]
                    for x in comments:
                        temp3=temp3+[all_comments[x]]
                        
                    module=Module(temp_module_code,"$$@$$".join([x[0] for x in temp3]))
                    print("\n\nin modules\n\n",module.module_code,module.comments)
                    total_modules_in_script.append(module)
                else:
                    comments=[]
            comments=[]
            code_lines=[]
            comments.append(i)
            
        elif(count_spaces(script_lines[i-1])==0 and len(script_lines[i-1])>0):
            code_lines.append(i)
            if i in all_comments.keys():
                comments.append(i)
            
        for temp in module_lines:
            
            if i>=temp[0] and i<=temp[1]:
                print("temp=",temp," i= ",i," comments =",comments)
                if len(comments)!=0:
                    temp2=temp[0] if temp[0]<comments[0] else comments[0]
                else:
                    temp2=temp[0]
                temp_module_code="\n".join(script_lines[temp2-1:temp[1]])
                i=temp[1]
                if (not(temp[1]-temp[0]>3)) or not(check_module_code(temp_module_code)):
                    comments=[]
                    break
                    
                #print("\nmodule\n")

                for x in range(temp[0],temp[1]+1):
                    if x in all_comments.keys():
                        comments.append(x)
                print("comments = ",comments)
                #print(temp_module_code)
                temp3=[]
                for x in comments:
                    temp3=temp3+[all_comments[x]]
                if(len(temp3)==0):
                    comments=[]
                    break
                    
                comments=[]
                #print("temp3 = ",temp3)
                module=Module(temp_module_code,"$$@$$".join([x[0] for x in temp3]))
                #print("\n\nin modules\n\n",module.module_code,module.comments)
                #print("\n\nin modules\n\n",module)
                total_modules_in_script.append(module)
        i=i+1
    return total_modules_in_script
                
                

    

code = ['##### GETTING COMMANDLINE PARAMETERS #####', '', 'parser = argparse.ArgumentParser()', "parser.add_argument('--serverName')", "parser.add_argument('--username')", "parser.add_argument('--password')"] 
 length= 6
parser. here2
False


In [90]:
temp="""A=b+"alert"
output = {
            "retCode": "0",
            "status": "Low",
            "result": totalUsedPercent + "%",
            "retDesc": "Used Space is Less than the Thershold Value"
        }"""

def get_output_vars(codeblock):
    """ code to check the variable output declaration in the codesnippet and extract if any useful string is present"""
    data=ast.parse(codeblock)
    #print(ast.dump(data))
    node_found=[]
    temp=[]
    for node in ast.walk(data):
        if isinstance(node, ast.Assign) and hasattr(node,'targets'):
            if hasattr(node.targets[0],'id') and node.targets[0].id.lower()=="output":
                node_found.append(node.value)
        
    if len(node_found)>0:
        print("node found, values are")
        for temp3 in node_found:
            for node in ast.walk(temp3):
                if isinstance(node,ast.Constant) and isinstance(node.value, str):
                    temp2=node.value.lower()
                    temp2=re.sub('[^a-zA-Z0-9?\s]',' ',temp2)
                    temp2=' '.join( [w for w in temp2.split() if len(w)>1] )
                    if(len(temp2.split())>=2):
                        temp.append(temp2)
    return temp
               
get_output_vars(temp)

node found, values are


['used space is less than the thershold value']

In [91]:

def parse_datamodule(data):
    """ function to find the dependent variables and functions of a particular code snippet """
    #print(data)
    data=ast.parse(data)
    #print(ast.dump(data))
    load=[]
    check=[]
    stored=[]
    Exceptions=[]
    Object_Names=[]
    def_Function_Names=[]
    Imports=[]
    Import_code={}
    Def_Func_code={}
    for node in ast.walk(data):
        if isinstance(node, ast.Name):
               check.append(node.id)
        if isinstance(node, ast.Name) and isinstance(node.ctx, eval("ast.Store")):
            stored.append(node.id)
        if isinstance(node, ast.Name) and isinstance(node.ctx, eval("ast.Load")):
            if node.id not in stored and node.id not in load:
                load.append(node.id)
        if isinstance(node,ast.ExceptHandler):
            if hasattr(node.type,'id'):    
                Exceptions.append(node.type.id)
            if hasattr(node,'name'):
                Exceptions.append(node.name)
        if isinstance(node,ast.Call):
            #print("node function",node.func,node.args)
            if isinstance(node.func,ast.Attribute) and isinstance(node.func.value,ast.Name):
                #print("has attribute",node.func.value.id)
                if node.func.value.id not in Object_Names:
                    Object_Names.append(node.func.value.id)
            if isinstance(node.func,ast.Name):
                #print("has name",node.func.id)
                def_Function_Names.append(node.func.id)
             
         
        if isinstance(node,ast.FunctionDef):
            print("check here function def")
            print(node.name)
            print(astunparse.unparse(node))
            Def_Func_code[node.name]=astunparse.unparse(node)
          
            
        if isinstance(node,ast.Import):
            for imp in node.names:
                if imp.asname is not None:
                    #print("module name = {}, alias = {}".format(imp.name, imp.asname))
                    Imports.append(imp.asname)
                    Import_code[imp.asname]=astunparse.unparse(node)
                else:
                    Imports.append(imp.name)
                    Import_code[imp.name]=astunparse.unparse(node)
                    #print("module name = {}".format(imp.name))
                    
        if isinstance(node,ast.ImportFrom):
            for imp in node.names:
                if imp.asname is not None:
                    Imports.append(imp.name)
                    Imports.append(imp.asname)
                    Import_code[imp.asname]=astunparse.unparse(node)
                    #print("module = {}\nname = {}\nalias = {}\nlevel = {}\n".format(node.module, imp.name, imp.asname, node.level))
                else:
                    Imports.append(imp.name)
                    Import_code[imp.name]=astunparse.unparse(node)
                    #print("module = {}\nname = {}\nlevel = {}\n".format(node.module, imp.name, node.level))
                
    load=list(set(load))
    stored=list(set(stored))
    Exceptions=list(set(Exceptions))
    Object_Names=list(set(Object_Names))
    def_Function_Names=list(set(def_Function_Names))
    Imports=list(set(Imports))

    Dependent_variable=list(set(load+Object_Names)-set(stored+Exceptions+def_Function_Names+Imports))
    
    

    #print(load)
    #print(stored)
    #print(Exceptions)
    #print(Object_Names)
    #print(def_Function_Names)
    #print(Imports)
    #print(Dependent_variable)
    return Dependent_variable,Import_code,Def_Func_code,def_Function_Names
    
          
def prep_cmt(cmt_lst):
    comment=''
    for i in cmt_lst:
        cmnt = re.sub('\s*#+\s*','',i)
        cmnt = re.sub('\s*"+\s*','',cmnt)
        print(cmnt)
        cmnt = cmnt.lower()
        comment += cmnt + "."
    return comment

def function_check(code):
    """ to check whether the code is function definition """
    code = re.sub('\s*#+\s*#*','',code)
    code = re.sub('\s*"+\s*','',code)
    def_var="def"
    for x in code.splitlines():
        if "\n" in x:
            continue
        elif def_var in x:
            return True
        elif len(x)>0:
            return False
    return False


def get_funcname(cmt):
    """ to create a function name based on the comments"""
    stop_words = set(stopwords.words('english'))
    cmt = cmt.replace("$$@$$",'')
    sentence = " ".join(cmt.split(" "))
    word_tokens = sentence.split(" ")
    filtered_sentence=list()
    for w in word_tokens:  
        if w not in stop_words:
            filtered_sentence.append(w)

    cmt_len = len(filtered_sentence)
    if cmt_len <= 3:
        #print('_'.join(filtered_sentence))
        return '_'.join(filtered_sentence)
    else:
        #print('_'.join(filtered_sentence[:3]))
        return '_'.join(filtered_sentence[:3])

def printdata(desc,script, code, cmnt,filepath,sheet,row):
    """to write the separated code snippet in to excel along with their dependencies"""
    discard,Full_File_Imports,Full_File_Def_funcs,discard2=parse_datamodule(script)
    print("\n\n\n\ check here \n\n",Full_File_Imports)
    print(Full_File_Def_funcs)
    print("\n\n")
    code_str=''
    arg_list,discard1,discard2,Function_Calls = parse_datamodule(code)
    print("\nInput Arguments:")
    print(arg_list)
    
    temp1=[]
    for x in arg_list:
        if x in Full_File_Imports:
            arg_list.remove(x)
            temp1.append(Full_File_Imports[x])    
    
    
    print("")
    temp=[]
    for x in Function_Calls:
        if x in Full_File_Def_funcs:
            temp.append(Full_File_Def_funcs[x])
        elif x in Full_File_Imports:
            temp1.append(Full_File_Imports[x])
    
    temp1=list(set(temp1))
    temp=list(set(temp))
    
    Full_File_Imports='\n'.join(temp1)
    
    Full_File_Def_funcs='\n'.join(temp)
    
    #desc = get_description(script)
    #cmt_txt = prep_cmt(cmnt)
    #print(cmt_txt)
    #doc_str = cmt_txt+"."+desc
    #print("\nDocument String:",)
    #print(doc_str)
    
    args=', '.join(arg_list)
    #print(code)
    print("\nCore Function:")
    fn_exist = function_check(code)
    #print(fn_exist)
    if fn_exist:
        print("Function already defined")
        code_str=code
    else:
        f_name = get_funcname(cmnt) #ignore this
        func = "def "+"$$@$$"+"("+args+"):\n"
        #print(func)
        code_str += func+"\n"
        f_code = code
        #print(f_code)
        for i in f_code.split('\n'):
            if re.match(r'^\s*$', i):
                continue
            else:
                #print("\t"+i)
                code_str+="\t"+i+"\n"
    print(code_str)
    temp=get_output_vars(code)
    if(len(temp)>0):
        cmnt=cmnt+"$$@$$"+"$$@$$".join(temp)
    sheet.cell(row,1).value=filepath
    sheet.cell(row,2).value=code_str
    sheet.cell(row,3).value=desc
    sheet.cell(row,7).value=code
    sheet.cell(row,8).value=Full_File_Imports
    sheet.cell(row,9).value=Full_File_Def_funcs
    sheet.cell(row,10).value=args
    sheet.cell(row,11).value=cmnt
    
    print("\n")     
    
    

In [92]:
workbook = Workbook()
sheet = workbook.active
row=1

In [93]:
count2=1
##### loop through each file in the given folder path#####
for filename in os.scandir(path):
        #print(filename)
        script_modules=[]
        if filename.is_file():
            fptr=open(filename.path,"r",encoding="utf-8")
            #fptr=open("489_DATABASE_ORACLE_FETCH_INSTANCES_WITH_ORACLE_HOME.py",'r')
            script=fptr.read()
            #print(script)
            #script='r'+script # adding r in the start of code to escape the unicode error which is present in the docstring
            data=ast.parse(script)
            doc_string=ast.get_docstring(data)
            code_description=""
            if doc_string is not None:
                code_description=get_description(doc_string)
            
            #print("script = ",script)
            comments=get_comment_line_nos(script)
            print("comments = ",comments)
            module_lines=separate_modules(script)
            print("module_lines = ",module_lines)
            
            #print(script)
            #print("\n-----------------inside finding the module-------------------\n")
            script_modules=script_modules+create_module(comments,module_lines,script)
            
            print("\n-------modules-------\n")
            for x in script_modules:
                #print("\n----------modulee in------- ")
                #print(x.module_code,"\ncomments\n",x.comments,"\n\n")
                #print("\n-------modules out-------\n")
                print(x)
                printdata(code_description,script, x.module_code, x.comments,filename.path,sheet,row)
                row=row+1
            #print(script_modules)
            count2=count2+1
            
print(count2,row)
            


over here
over here
over here
over here
over here
{4: ['this function just adds two numbers and performs math ceil and floor', 'comment'], 10: ['this logic checks which number is smallest', 'comment'], 19: ['This function checks if the given input a is less than b or not', 'comment'], 25: ['to check the smallest of all three numbers', 'comment'], 28: ['we have to check smallest of other two numbers', 'comment']}
comments =  {4: ['this function just adds two numbers and performs math ceil and floor', 'comment'], 10: ['this logic checks which number is smallest', 'comment'], 19: ['This function checks if the given input a is less than b or not', 'comment'], 25: ['to check the smallest of all three numbers', 'comment'], 28: ['we have to check smallest of other two numbers', 'comment']}
[[3, 8], [13, 16], [18, 23], [27, 37], [20, 23], [29, 32], [33, 37]]
[[3, 8], [13, 16], [18, 23], [27, 37], [20, 23], [29, 32], [33, 37]]
here [20, 23]
here [29, 32]
here [33, 37]
module_lines =  [[3, 8], [

In [94]:
workbook.save(filename="Split_Data1.xlsx")